# panel dashboard with rollups

> This notebook uses panel to make dashboards

In [1]:
# |default_exp rollups_blobs

In [2]:
# | export
import holoviews as hv
import hvplot.pandas
import panel as pn
import polars as pl
import polars.selectors as cs
import jupyter_black


from panel.widgets import Tabulator

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")
jupyter_black.load()
pn.extension("tabulator", template="material", sizing_mode="stretch_width")

In [3]:
explore_df = (
    pl.read_parquet("../data/rollups/*.parquet")
    .filter(pl.col("block_datetime") >= pl.datetime(2023, 1, 1))
    .sort(by="block_number")
    .with_columns(
        (pl.col("avg_base_fee_daily") * pl.col("gas_used") / 10**18).alias(
            "tx_gas_cost_avg_daily"
        ),
        (pl.col("avg_base_fee_hourly") * pl.col("gas_used") / 10**18).alias(
            "tx_gas_cost_avg_hourly"
        ),
        (pl.col("avg_base_fee_minute") * pl.col("gas_used") / 10**18).alias(
            "tx_gas_cost_avg_minute"
        ),
    )
    .with_columns(
        # ITM when positive, OTM when negative
        (pl.col("tx_gas_cost") - pl.col("tx_gas_cost_avg_daily")).alias(
            "tx_gas_cost_delta_daily"
        ),
        (pl.col("tx_gas_cost") - pl.col("tx_gas_cost_avg_hourly")).alias(
            "tx_gas_cost_delta_hourly"
        ),
        (pl.col("tx_gas_cost") - pl.col("tx_gas_cost_avg_minute")).alias(
            "tx_gas_cost_delta_minute"
        ),
    )
)

In [4]:
def summarize_dataset(df: pl.DataFrame) -> pn.Column:
    """
    Summarizes the dataset block and date range, and number of transactions.
    """
    # cryo txs
    dataset_info = (
        (df.select("block_number", "block_datetime")).unique().sort(by="block_number")
    )

    # Formatting the information as strings
    block_range_info = (
        f"block range: {dataset_info['block_number'][0]:,.0f} to {dataset_info['block_number'][-1]:,.0f} = "
        f"{(dataset_info['block_number'][-1] - dataset_info['block_number'][0]):,.0f} range"
    )

    blocks_dataset_info = f"blocks in dataset: {dataset_info['block_number'].unique().len():,.0f}, total txs: {df.shape[0]:,.0f}"

    block_date_range_info = f"block date range: {dataset_info['block_datetime'][0]} to {dataset_info['block_datetime'][-1]}"

    # Markdown intro:
    markdown_intro = """
    ### Setup & Methodology
    * **Labeling** We label the sequencer submitter addresses from each L2 and analyze how much gas they consume in blocks. We measure the block utilization behavior of L2s pre EIP-4844 and then extrapolate, from historical demand, how saturated the initial data blob market will be. 
    * **Preconfirmations** are critical to securing future blockspace at less volatile gas prices. A preconfirmation is the right, but not obligation, to reserve future blockspace at the breakeven price. A future contract "in the money" would be when the future block base gas > breakeven gas price and being "out of money" would be whent he future block base gas < breakeven gas price and the future expires worthless. 
    """

    # Create Markdown or HTML widgets
    intro_widget = pn.pane.Markdown(markdown_intro)
    block_range_widget = pn.pane.Markdown(block_range_info)
    blocks_dataset_widget = pn.pane.Markdown(blocks_dataset_info)
    block_date_range_widget = pn.pane.Markdown(block_date_range_info)

    # Create the dashboard layout
    return pn.Column(
        intro_widget, block_range_widget, blocks_dataset_widget, block_date_range_widget
    )

In [5]:
summarize_dataset(explore_df)

BokehModel(combine_events=True, render_bundle={'docs_json': {'af7875f9-9a2a-4708-93ad-ad0cbdc3b58d': {'version…

In [6]:
summarize_dataset(explore_df).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'8d006263-5c40-43b0-bae8-a3b6cfe2a44b': {'version…

In [7]:
from holoviews import opts

In [24]:
def calldata_size_transform(df: pl.DataFrame, pivot_column: str) -> pl.DataFrame:
    # pivot dataframe by sequencer_names column
    explore_df_pivot: pl.DataFrame = (
        (
            df.select(
                "block_number",
                "block_datetime",
                "gas_price_gwei",
                "sequencer_names",
                "block_calldata_kbytes",
                "n_input_bytes",
            )
            .with_columns(
                # calculate the block_gas ratio and convert bytes to kilobytes
                (
                    pl.col("block_calldata_kbytes") / \
                    pl.col("gas_price_gwei") / 10**3
                ).alias("calldata_mb_block_gas_ratio"),
                (pl.col("n_input_bytes") / pl.col("gas_price_gwei") / 10**6).alias(
                    "sequencer_mb_gas_ratio"
                ),
            )
            .group_by("block_number", "sequencer_names")
            .agg(
                pl.col("block_datetime").first(),
                pl.col("block_calldata_kbytes").first(),
                pl.col("gas_price_gwei").first(),
                pl.col("calldata_mb_block_gas_ratio").mean(),
                pl.col("sequencer_mb_gas_ratio").mean(),
                pl.col("n_input_bytes").sum().alias("sequencer_total_bytes"),
            )
            .with_columns(
                (
                    pl.col("sequencer_total_bytes") / \
                    pl.col("block_calldata_kbytes")
                ).alias("sequencer_proportion_of_calldata"),
                (pl.col("sequencer_total_bytes") / 10**6).alias(
                    "sequencer_total_mbytes"
                ),
            )
            .select(
                "block_number",
                "block_datetime",
                "sequencer_names",
                "calldata_mb_block_gas_ratio",
                "block_calldata_kbytes",
                "gas_price_gwei",
                "sequencer_mb_gas_ratio",
                "sequencer_total_mbytes",
            )
            .pivot(
                index="block_datetime",
                columns="sequencer_names",
                values=pivot_column,
                aggregate_function="mean",
            )
            .fill_null(0)
            .group_by(pl.col("block_datetime").dt.date())
            .sum()
        )
        .select("block_datetime", cs.numeric().round(2))
        .sort(by="block_datetime")
    )
    return explore_df_pivot


def create_calldata_posted_chart(
    df: pl.DataFrame, title: str, sequencers: list[str], cmap: list[str]
):
    """
    creates bar charts for calldata_size_transform() - calldata_mb_block_gas_ratio and sequencer_proportion_of_calldata
    """
    nested_col_list = ["block_datetime", sequencers]
    cols = [
        item
        for sublist in nested_col_list
        for item in (sublist if isinstance(sublist, list) else [sublist])
    ]

    df_pandas = (
        df.select(cols).to_pandas().set_index(
            "block_datetime").resample("M").mean()
    )

    df_pandas.index = df_pandas.index.strftime("%b %Y")

    return df_pandas.hvplot.bar(
        x="block_datetime",
        xlabel="datetime",
        y=sequencers,
        ylabel="calldata (in MB)",
        stacked=True,
        rot=90,
        title=title,
        width=600,
        height=700,
        cmap=cmap,
        shared_axes=False,
        legend="top_left",
    )


def create_calldata_gas_ratio_chart(
    df: pl.DataFrame, title: str, sequencers: list[str], cmap: list[str]
):
    """
    creates bar charts for calldata_size_transform() - calldata_mb_block_gas_ratio and sequencer_proportion_of_calldata
    """
    nested_col_list = ["block_datetime", sequencers]
    cols = [
        item
        for sublist in nested_col_list
        for item in (sublist if isinstance(sublist, list) else [sublist])
    ]

    df_pandas = (
        df.select(cols).to_pandas().set_index(
            "block_datetime").resample("M").mean()
    )

    df_pandas.index = df_pandas.index.strftime("%b %Y")

    return df_pandas.hvplot.bar(
        x="block_datetime",
        xlabel="datetime",
        y=sequencers,
        ylabel="Calldata Cost per MB (in Gwei)",
        rot=90,
        title=title,
        width=1000,
        height=600,
        cmap=cmap,
        shared_axes=False,
        # stacked=True,
        legend="top_left",
    )

In [25]:
# dataframes for charting
calldata_posted_df: pl.DataFrame = calldata_size_transform(
    explore_df, pivot_column="sequencer_total_mbytes"
)

# ! NOT USED
# calldata_gas_ratio_df: pl.DataFrame = calldata_size_transform(
#     explore_df, pivot_column="calldata_mb_block_gas_ratio"
# )

calldata_gas_ratio_df: pl.DataFrame = calldata_size_transform(
    explore_df, pivot_column="sequencer_mb_gas_ratio"
)

# dropdown list
sequencer_names_list: list[str] = sorted(
    explore_df["sequencer_names"].unique().to_list()
)

cmap_list: list[str] = [
    "red",
    "green",
    "blue",
    "orange",
    "purple",
    "cyan",
    "magenta",
    "yellow",
]

In [26]:
multi_select = pn.widgets.MultiSelect(
    name="Sequencers",
    size=10,
    options=sequencer_names_list,
    value=sequencer_names_list,
)

title_one = "Calldata Posted Per Sequencer (MB)"
title_two = "Average Sequencer Calldata Cost (MB/gwei)"

calldata_posted_and_gas_ratio_panel = pn.Row(
    create_calldata_posted_chart(
        calldata_posted_df,
        title=title_one,
        sequencers=sequencer_names_list,
        cmap=cmap_list,
    ),
    create_calldata_gas_ratio_chart(
        calldata_gas_ratio_df,
        title=title_two,
        sequencers=sequencer_names_list,
        cmap=cmap_list,
    ),
)

entire_panel = pn.Column(multi_select, calldata_posted_and_gas_ratio_panel)


def update_bar_chart(event):
    entire_panel[1][0].object = create_calldata_posted_chart(
        calldata_posted_df,
        title=title_one,
        sequencers=multi_select.value,
        cmap=cmap_list,
    )
    entire_panel[1][1].object = create_calldata_gas_ratio_chart(
        calldata_gas_ratio_df,
        title=title_two,
        sequencers=multi_select.value,
        cmap=cmap_list,
    )


multi_select.param.watch(update_bar_chart, "value")

entire_panel.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'107939d9-dc5b-4327-952f-eeb66ef62e0a': {'version…

In [11]:
# WORK I PROGRESS

In [12]:
def create_calldata_cost_chart(df: pl.DataFrame) -> pn.Column:
    chart_df: pl.DataFrame = (
        df.select(
            "block_number",
            "block_datetime",
            "gas_price_gwei",
            "sequencer_names",
            "block_calldata_kbytes",
            "n_input_bytes",
        )
        .with_columns(
            # calculate the block_gas ratio and convert bytes to kilobytes
            (pl.col("block_calldata_kbytes") / pl.col("gas_price_gwei")).alias(
                "calldata_block_gas_ratio"
            ),
            (pl.col("n_input_bytes") / 10**3).alias("n_input_kbytes"),
        )
        .group_by("block_number", "sequencer_names")
        .agg(
            pl.col("block_datetime").first(),
            pl.col("block_calldata_kbytes").first(),
            pl.col("gas_price_gwei").first(),
            pl.col("calldata_block_gas_ratio").mean(),
            pl.col("n_input_kbytes").sum().alias("sequencer_total_bytes"),
        )
        .with_columns(
            (pl.col("sequencer_total_bytes") / pl.col("block_calldata_kbytes")).alias(
                "sequencer_proportion_of_calldata"
            )
        )
        .filter(
            pl.col("sequencer_names") == "arbitrum"
        )  # TODO 1/23/24 - turn into a widget
        .select(
            "block_number",
            "block_datetime",
            "sequencer_names",
            "calldata_block_gas_ratio",
            "block_calldata_kbytes",
            "gas_price_gwei",
        )
        # add a group by datetime day?
        .sort(by="block_number", descending=True)
    )

    markdown_txt: str = """
                These metrics will set a baseline for the pattern between how much the calldata is boosting the gas price and block size.
                * `block_calldata_kbytes` - amount of calldata in a block
                * `sequencer_total_bytes` - amount of calldata contributed by a single sequencer tx
                * `calldata_block_gas_ratio` - how expensive the calldata was in relation to the block gas price
                """

    txt_widget: pn.pane.Markdown = pn.pane.Markdown(markdown_txt)
    calldata_usage_chart: pn.Column = pn.Column(
        chart_df.plot.line(
            x="block_datetime",
            y="block_calldata_kbytes",
            height=500,
            width=600,
            title="Arbitrum calldata used per block (kb)",
        ),
    )
    calldata_gas_ratio_chart: pn.Column = pn.Column(
        chart_df.plot.line(
            x="block_datetime",
            y="calldata_block_gas_ratio",
            height=500,
            width=600,
            title="Arbitrum calldata/gas per block",
        ),
    )

    chart_structure: pn.Row = pn.Row(calldata_usage_chart, calldata_gas_ratio_chart)

    # Create the dashboard layout
    return pn.Column(txt_widget, chart_structure)

In [13]:
create_calldata_cost_chart(explore_df).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'69decaa4-aa73-423e-bd11-4a4ca53a64b0': {'version…

### Aggregate Sequencer Stats

In [14]:
def agg_sequencer_stats(df: pl.DataFrame):
    """
    Prints aggregate sequencer statistics
    """
    agg_calldata: pl.DataFrame = (
        df.group_by(["block_number", "sequencer_names"])
        .agg(
            pl.len().alias("n_txs"),
            pl.col("gas_used_block").sum().alias("sequencer_gas_used"),
            pl.col("tx_gas_cost").sum().alias("tx_gas_cost_sum_eth"),
            pl.col("n_input_bytes").sum().alias("sequencer_n_input_bytes_sum"),
            pl.col("n_input_zero_bytes")
            .sum()
            .alias("sequencer_n_input_zero_bytes_sum"),
            pl.col("n_input_nonzero_bytes")
            .mean()
            .alias("sequencer_n_input_nonzero_bytes_mean"),
            pl.col("n_rlp_bytes").sum().alias("sequencer_n_rlp_bytes_sum"),
            pl.col("gas_price_gwei").mean().alias("sequencer_gas_price_mean"),
            # ! Currently not used - selecting columns that are the same for all rows in the group
            pl.col("base_fee_per_gas_gwei").first(),
            pl.col("block_datetime").first(),
            pl.col("block_encoded_kbytes").first(),
            pl.col("block_calldata_kbytes").first(),
        )
        .group_by(["sequencer_names"])
        .agg(
            pl.col("block_number").len().alias("n_blocks").cast(pl.Float64).round(0),
            pl.col("n_txs").sum().cast(pl.Float64).round(0),
            pl.col("tx_gas_cost_sum_eth").sum().round(2),
            pl.col("sequencer_n_input_bytes_sum").sum(),
            pl.col("sequencer_n_input_zero_bytes_sum").sum(),
            pl.col("sequencer_n_input_nonzero_bytes_mean").mean().round(2),
            pl.col("sequencer_n_rlp_bytes_sum").sum(),
            pl.col("sequencer_gas_price_mean").mean().round(2),
        )
        .with_columns(
            (pl.col("sequencer_n_input_bytes_sum") / 10**6)
            .alias("n_input_mbytes_sum")
            .round(2),
            (pl.col("sequencer_n_rlp_bytes_sum") / 10**6)
            .alias("n_rlp_mbytes_sum")
            .round(2),
        )
        .drop("sequencer_n_input_bytes_sum", "sequencer_n_rlp_bytes_sum")
        .with_columns(
            (pl.col("n_rlp_mbytes_sum") / pl.col("tx_gas_cost_sum_eth"))
            .alias("mb_data_per_eth")
            .round(2),
        )
        .sort(by="mb_data_per_eth", descending=True)
    ).select(
        "sequencer_names",
        "n_blocks",
        "n_txs",
        "tx_gas_cost_sum_eth",
        "mb_data_per_eth",
        "n_input_mbytes_sum",
    )

    # cost adjustment
    agg_tx_costs: pl.DataFrame = (
        df.select(
            "block_number",
            "sequencer_names",
            "base_fee_per_gas_gwei",
            "avg_base_fee_daily",
            "avg_base_fee_minute",
            "gas_price_gwei",
            "gas_used",
            "tx_gas_cost",
            "tx_gas_cost_avg_daily",
            "tx_gas_cost_avg_hourly",
            "tx_gas_cost_avg_minute",
        )
        .group_by("sequencer_names")
        .agg(
            pl.col("tx_gas_cost_avg_daily")
            .sum()
            .round(2)
            .alias("tx_gas_cost_daily_sum"),
            pl.col("tx_gas_cost_avg_hourly")
            .sum()
            .round(2)
            .alias("tx_gas_cost_hourly_sum"),
            pl.col("tx_gas_cost_avg_minute")
            .sum()
            .round(2)
            .alias("tx_gas_cost_avg_minute_sum"),
        )
    )

    return (
        agg_calldata.join(agg_tx_costs, on="sequencer_names", how="left")
        .rename(
            {
                "sequencer_names": "sequencer",
                "mb_data_per_eth": "mb_per_eth",
                "n_input_mbytes_sum": "total_mbytes",
                "tx_gas_cost_sum_eth": "tota_cost_eth",
                "tx_gas_cost_daily_sum": "tx_cost_daily_delta_eth",
                "tx_gas_cost_hourly_sum": "tx_cost_hourly_delta_eth",
                "tx_gas_cost_avg_minute_sum": "tx_cost_minute_delta_eth",
            }
        )
        .sort(by="n_txs", descending=True)
    )

In [15]:
agg_sequencer_stats(explore_df)

sequencer,n_blocks,n_txs,tota_cost_eth,mb_per_eth,total_mbytes,tx_cost_daily_delta_eth,tx_cost_hourly_delta_eth,tx_cost_minute_delta_eth
str,f64,f64,f64,f64,f64,f64,f64,f64
"""starknet""",481141,1133789,13313.55,0.31,3949.56,11586.72,12767.19,12842.95
"""optimism""",605981,608363,13866.33,0.18,2368,13090.28,13775.29,13830.27
"""arbitrum""",334386,453611,25985.48,0.08,1859.08,24566.64,25688.88,25830.82
"""linea""",141642,376242,9889.82,0.33,3203.21,9681.24,9919.48,9858.52
"""base""",298757,301088,4002.66,0.69,2733.03,3779.38,3979.7,3997.06
"""scroll""",58056,58150,2285.81,0.76,1724.32,2169.24,2271.91,2283.85
"""polygon_zkevm""",51297,51297,1510.68,0.35,522.43,1408.94,1497.53,1469.46
"""mantle""",15616,15616,1863.97,0.84,1565.15,1757.51,1851.34,1861.75


In [16]:
# Create a Tabulator widget
tabulator = Tabulator(
    agg_sequencer_stats(explore_df).to_pandas(), pagination="local", page_size=10
)

# Create the Panel layout
interactive_agg_sequencer_widget = pn.Column(
    pn.pane.Markdown("Aggregate Sequencer Stats"), tabulator
).servable()

interactive_agg_sequencer_widget

BokehModel(combine_events=True, render_bundle={'docs_json': {'f91f11c6-20be-4abf-b729-33b3d4e9c756': {'version…

In [17]:
sequencer_gas_sum = explore_df.group_by("block_number", "sequencer_names").agg(
    [
        pl.col("gas_used")
        .sum()
        .alias("sequencer_gas_used_sum")
        .cast(pl.Float64)
        .round(0),
        pl.count().alias("n_blocks"),
    ]
)

# Create a histogram plot
sequencer_histogram = sequencer_gas_sum.filter(
    pl.col("sequencer_gas_used_sum") < 10**8
).plot.hist(
    y="sequencer_gas_used_sum",
    by="sequencer_names",
    bins=50,
    stacked=True,
    height=500,
    width=700,
    xlabel="Sequencer Gas Used (in millions of Gas)",  # Custom X-axis label
    ylabel="Sequencer Gas Count",  # Custom Y-axis label
    title="Sequencer Gas Usage per block",
)

# TITLE - Gas usage per Sequencer. This chart shows the individual gas usage for each sequencer in a block
sequencer_scatter = (
    (
        explore_df.group_by("block_number", "sequencer_names").agg(
            (pl.col("gas_used").sum() / pl.col("gas_used_block").first()).alias(
                "sequencer_gas_used_pct"
            ),
            (pl.col("gas_used_block").first()).alias("total_gas_in_block"),
        )
    )
    .sort(by="sequencer_gas_used_pct", descending=True)
    .plot.scatter(
        x="sequencer_gas_used_pct",
        y="total_gas_in_block",
        by="sequencer_names",
        height=500,
        width=700,
        xlabel="Sequencer Gas % Used of Total Block Size",  # Custom X-axis label
        ylabel="Block Size (in millions of Gas)",  # Custom Y-axis label
        title="Sequencer Gas Usage per block",
    )
)

/tmp/ipykernel_56766/3975152810.py:8: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("n_blocks"),


In [18]:
# Create Markdown or HTML widgets
histogram_warning = pn.pane.Markdown("Warning - Histogram isn't stacked yet").servable()
histogram_warning

BokehModel(combine_events=True, render_bundle={'docs_json': {'d04cf307-ded3-4631-a58e-85653ecea0d5': {'version…

In [19]:
pn.Row(sequencer_histogram, sequencer_scatter).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'55cdc28e-3328-4db1-881c-89387caccdb4': {'version…